In [111]:
import random
import re
from urllib.parse import urlparse, parse_qs, parse_qsl, quote

import pandas as pd
import requests

from tqdm import tqdm

from bs4 import BeautifulSoup
import bs4

URL_ROOT = 'https://en.wiktionary.org'

soups = {}
def get_soup(path):
    if path in soups:
        return soups[path]
    
    res = requests.get(f"{URL_ROOT}{path}")
    soup = BeautifulSoup(res.text)
    soups[path] = soup
    return soup

# e.g. get_category_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_category_pages(start_path):
    paths = [start_path]    
    next_path = start_path
    soup = get_soup(next_path)
    next_link = soup.find('a', string='next page')
    while next_link:
        next_path = next_link['href']
        paths.append(next_path)
        soup = get_soup(next_path)
        next_link = soup.find('a', string='next page')
    return paths

# e.g. get_lemma_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_lemma_pages(path):
    soup = get_soup(path)
    return [{'href': a['href'], 'title': a['title']} for a in soup.css.select('div#mw-pages li a')]

def get_definitions(headword, languages=['ajp', 'apc']):
    lemma = headword.strong.get_text()
    language = headword.strong.attrs['lang']
    headline = headword.find_previous(class_='mw-headline').string
    print(lemma)

    def_ol = headword.find_next_sibling('ol')
    
    results = []
    for li in def_ol.select('li'):
        result = {'lemma': lemma, 'language': language, 'headline': headline}
        definition = ''
        for s in li.strings:
            if 'dl' in list(p.name for p in s.parents):
                break
            definition += s
        result['definition'] = definition.strip()
        if li.dl:
            detail = '\n'.join(e.get_text() for e in  li.dl.find_all(class_=re.compile('^e')))
            result['detail'] = detail.strip()
        results.append(result)
    return results



In [201]:
pd.set_option('display.max_colwidth', 250)

In [338]:
# Category:Armenian_terms_with_usage_examples
# Category:Old_Armenian_terms_with_usage_examples

lps = ([lp for cp in get_category_pages('/wiki/Category:Armenian_terms_with_usage_examples')
    for lp in get_lemma_pages(cp)])
lemmas = pd.DataFrame(lps)
lemmas

,href,title
0,/wiki/%D5%A1,ա
1,/wiki/%D5%A1%CC%88,ա̈
2,/wiki/%D5%A1%CC%8A,ա̊
3,/wiki/%D5%A1%D5%A2%D5%A5%D5%A9%D5%A1%D5%BD%D5%B8%D6%82%D5%B6%D5%AF,աբեթասունկ
4,/wiki/%D5%A1%D5%A3%D5%BC%D5%A1%D5%BE,ագռավ
...,...,...
1789,/wiki/%D5%96%D6%80%D5%B8%D6%82%D5%B6%D5%A6%D5%A5,Ֆրունզե
1790,/wiki/%D5%9B,՛
1791,/wiki/%D5%9E,՞
1792,/wiki/%D6%89,։


In [339]:
def getLanguageSections(soup):
    headings = soup.select('div#bodyContent div.mw-heading2')
    
    heading_map = {}

    for heading in headings:
        language = heading.find('h2').text
        content = []
        for sib in heading.next_siblings:
            if sib == '\n':
                continue
            if type(sib) == bs4.element.Comment:
                continue
            if sib in headings:
                break
            content.append(sib)
        heading_map[language] = content
    return heading_map

def getSubSections(section):
    result = {}
    current_header = None
    current_subsection = bs4.Tag(name='div')
    for tag in section:
        if tag.name == 'div' and 'mw-heading' in tag.attrs.get('class', []):
            if current_header:
                result[current_header] = current_subsection
            current_header = tag.text.replace('[edit]', '')
            current_subsection = bs4.Tag(name='div')
        else:
            current_subsection.append(tag)
    if current_header:
        result[current_header] = current_subsection
    return result

In [388]:
entries = {}

for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas.sample(100).iterrows()]):
    sections = getLanguageSections(get_soup(href))
    if 'Armenian' not in sections:
        continue
    entries[title] = getSubSections(sections['Armenian'])

100%|██████████| 100/100 [01:08<00:00,  1.45it/s]


In [389]:
results = pd.DataFrame(entries).T

In [390]:
columns_by_freq = results.count().sort_values(ascending=False)
results = results[columns_by_freq.index] #[lambda x:x['Verb'].notna()]
columns_by_freq.to_frame().style.bar()

,0
Pronunciation,100
Etymology,91
Declension,72
Noun,48
Derived terms,33
Adjective,28
Alternative forms,26
Related terms,17
Adverb,14
Verb,14


In [391]:
def process_pronunciation(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return '\n'.join([li.text for li in tag.select('li') if not 'Audio' in li.text and not li.text == ''])

results['Pronunciation'] = results['Pronunciation'].apply(process_pronunciation)

In [392]:
def process_basic_text(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return tag.text.strip()


results['Etymology'] = results['Etymology'].apply(process_basic_text)

In [393]:
results.sample(5)

,Pronunciation,Etymology,Declension,Noun,Derived terms,Adjective,Alternative forms,Related terms,Adverb,Verb,...,Suffix,Antonyms,Conjugation,Letter,Prefix,Pronoun,Preposition,Particle,Proper noun,Conjunction
երբ,"(Eastern Armenian) IPA(key): /jeɾpʰ/, [jeɾpʰ]\n(Western Armenian) IPA(key): /jeɾp/, [jeɾpʰ]",Learned borrowing from Old Armenian երբ (erb),NaN,NaN,NaN,NaN,"[[[<span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%B5%D5%A5%D6%80%D5%A2&amp;action=edit&amp;redlink=1"" title=""յերբ (page does not exist)"">յերբ</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <s...",NaN,"[[[<strong class=""Armn headword"" lang=""hy"">երբ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">erb</span>, )], \n],...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-անք,"(Eastern Armenian) IPA(key): /ɑnkʰ/, [ɑŋkʰ]\n(Western Armenian) IPA(key): /ɑnk/, [ɑŋkʰ]",NaN,"[[\n, [<small> <i>i</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...",NaN,"[[[<div class=""CategoryTreeItem""><span class=""CategoryTreeBullet""><a aria-expanded=""false"" class=""CategoryTreeToggle"" data-ct-title=""Armenian_terms_suffixed_with_-անք_(collective)""></a> </span> <a href=""/wiki/Category:Armenian_terms_suffixed_with...",NaN,NaN,NaN,NaN,NaN,...,"[[[<strong class=""Armn headword"" lang=""hy"">-անք</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">-ankʻ</span>, )], \...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
բեռ,"(Eastern Armenian) IPA(key): /ber/, [ber]\n(Western Armenian) IPA(key): /per/, [pʰer]",From Old Armenian բեռն (beṙn).,"[[\n, [<small> <i>n</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...","[[[<strong class=""Armn headword"" lang=""hy"">բեռ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">beṙ</span>, )], \n],...","[[[<ul><li><span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%A2%D5%A5%D5%BC%D5%B6%D5%A1%D5%A3%D5%AB%D5%B6&amp;action=edit&amp;redlink=1"" title=""բեռնագին (page does not exist)"">բեռնագին</a></span> <span class=""mention-gloss-...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
հանդիման,"(Eastern Armenian) IPA(key): /hɑndiˈmɑn/, [hɑndimɑ́n]\n(Western Armenian) IPA(key): /hɑntiˈmɑn/, [hɑntʰimɑ́n]",From Old Armenian յանդիման (yandiman).,NaN,NaN,NaN,NaN,NaN,NaN,"[[[<strong class=""Armn headword"" lang=""hy"">հանդիման</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">handiman</span>...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
դույզն,"(Eastern Armenian) IPA(key): /ˈdujzən/, [dújzən]\n(Western Armenian) IPA(key): /ˈtujzən/, [tʰújzən]",Learned borrowing from Old Armenian դոյզն (doyzn).,NaN,NaN,NaN,"[[[<strong class=""Armn headword"" lang=""hy"">դույզն</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">duyzn</span>, ) (...","[[[<span class=""Armn"" lang=""hy""><a href=""/wiki/%D5%A4%D5%B8%D5%B5%D5%A6%D5%B6#Armenian"" title=""դոյզն"">դոյզն</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <sp